In [ ]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import numpy as np
%matplotlib inline 
from matplotlib import pyplot as plt
from Bio import SeqIO
import customfunc
import matrix

## Abricate
Run [abricate](https://github.com/tseemann/abricate) to search for antibiotic resistance genes, plasmid genes and virulence factors in several databases

In [ ]:
#!abricate ~/documentos/sentinel/genomasbrasil/*.gbff --db argannot --noheader --threads 12 > ~/documentos/github/ARG-Sentinel/spreadsheets/argannot_br.csv

In [ ]:
#!abricate ~/documentos/sentinel/genomasbrasil/*.gbff --db plasmidfinder --noheader --threads 12 > ~/documentos/github/ARG-Sentinel/spreadsheets/plasmidfinder_br.csv

## Load dataframes

In [ ]:
#Catalog of genomes colected from Brazil
catalog = pd.read_csv("~/documentos/github/ARG-Sentinel/spreadsheets/catalogo_teste.csv")
catalog.head()

In [ ]:
#ARG-Annot database genes
argannot = pd.read_csv("~/documentos/github/ARG-Sentinel/spreadsheets/argannot_br.csv", sep = '\t', names = ['file','sequence','start','end','strand','gene','coverage','coverage_map','gaps','coverage_pec','identity_perc','database','accession','product','resistance'])
argannot['file'] = argannot['file'].apply(lambda x: x.split('/')[-1])
argannot = customfunc.filldf(argannot,catalog)
argannot.head()

In [ ]:
#Plasmidfinder database genes
plasmidfinder = pd.read_csv("~/documentos/github/ARG-Sentinel/spreadsheets/plasmidfinder_br.csv",sep = "\t", names = ['file','sequence','start','end','strand','gene','coverage','coverage_map','gaps','coverage_pec','identity_perc','database','accession','product','resistance'])
plasmidfinder['file'] = plasmidfinder['file'].apply(lambda x: x.split('/')[-1])
plasmidfinder = customfunc.filldf(plasmidfinder, catalog)
plasmidfinder.head()

## Brazil resistome profile 

In [ ]:
argannot['class'] = argannot['gene'].apply(lambda x: x.split(')')[0]).apply(lambda y: y.split('(')[1]) #Creating the "class" column
profile = argannot[['organism','class']].set_index('organism')#take importa columns
profile = pd.get_dummies(profile).groupby(level = 0).sum()
profile.head()

In [ ]:
plt.figure(figsize = (12,10))
sns.heatmap(profile, cmap = "RdBu_r")
plt.title('Resistome profile from brazilian genomes', fontsize = 40, y = .8)
plt.colormaps()
plt.xticks(fontsize = 20, rotation = 45)
plt.yticks(fontsize = 15, fontstyle = 'italic')
plt.ylabel('')
#plt.savefig('heatmapmap_classes_argannot.png', bbox_inches = 'tight',dpi = 600)

## Retirar genomas para construção de filogenias

## Sets

## Diagrama de Veen: procurar por elementos moveis

## Encontrar plasmídeos

# Co-ocorrência entre patôgenos humanos e não humanos (ou bactérias de vida livre)
<p>
    Alguns estudos mostraram que bactérias patogênicas que não infectam humanos ou, ainda, as não potogênicas são potênciais reservatorórios de ARGs devido ao contato com antibióticos fora da área médica. <br /> 
    <b>Objetivo:</b> encontrar espécies de interesse clínico e checar se elas compartilham genes de resistência com grupos não patogênicos ou patogênos de outros animais/plantas.<br />
    <br />
<b>Lista de interesse: patôgenos ESKAPE +  E.coli</b>  
    <ul>
        <li> Enterococcus faecium   [Presente] </li>
        <li> Staphylococcus aureus  [Presente] </li>
        <li> Klebsiella pneumoniae  [Presente]</li>
        <li> Acinetobacter baumannii[Presente]</li>
        <li> Pseudomonas aeruginosa [Presente]</li>
        <li> Enterobacter spp       [Presente :cloacae, kobei e  hormaechei]</li>
        <li> Escherichia coli       [Presente]</li>
    </ul>
    
</p>

[ESKAPE](https://en.wikipedia.org/wiki/ESKAPE)

**Passo 1:** selecionar os patôgenos humanos de interesse

In [ ]:
#lista de patôgenos de interesse
eskape = ['Enterococcus faecium',
          'Staphylococcus aureus',
          'Klebsiella pneumoniae',
          'Acinetobacter baumannii',
          'Pseudomonas aeruginosa',
          'Enterobacter cloacae',
          'Enterobacter kobei',
          'Enterobacter hormaechei',
          'Escherichia coli']

**Passo 2:** encontrar ARGs presentes nos patógenos ESKAPE dos nossos genomas


**Passo 3:** checar a existência desses mesmoas ARGs em bactérias ambientais

# Como os ARGs se espalharam pelo Brasil?

<p style='text-align: justify;'> 
    Como pesquisar a movimentação dos ARGs pelo território nacional?<br />
    Preciso unir das informações: ano de coleta da amostra, coordenada e linhagens onde esses ARGs aparecem. <br />
    Figura resultante: um mapa do Brasil com grafo sobreposto.
</p>

In [ ]:
args_spread = argannot[['gene','coordenates','colection_date']].query("coordenates != 'Na'").query("colection_date != 'Na' ")


In [ ]:
args_spread = argannot[['gene','coordenates','colection_date']].query("coordenates != 'Na'").query("colection_date != 'Na' ")
args_spread.colection_date  = args_spread.colection_date.apply(lambda x: customfunc.SwitchMonths(x))
args_spread.head()



In [ ]:
#info sobre args_spread
print(f"Total de {len(args_spread)} ARGs coletados com coordenadas")
print(f"{len(set(args_spread.coordenates))} distintos")